In [ ]:
%run Lib_shortest_path_for_EV.ipynb

In [ ]:
def show_plot(elem_list,title='length histogram',xlabel='length in km',ylabel='frequency'):
    elem_list = list(map(int, elem_list))
    elem_list_size=len(elem_list)
    bins=round(math.sqrt(elem_list_size))
    plt.hist(elem_list,bins,histtype='bar',rwidth=1)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()


    elem_list_mean = mean(elem_list)
    elem_list_median = median(elem_list)
    elem_list_std=np.std(elem_list)
    elem_list_min=min(elem_list)
    elem_list_max=max(elem_list)

    print("mean : "+str(elem_list_mean))
    print("median : "+str(elem_list_median))
    print("min : "+str(elem_list_min))
    print("max : "+str(elem_list_max))
    print("STD : "+str(elem_list_std))


In [ ]:
def show_nodes_on_map(start_node,end_node,batteria=None,region_x=43.7711,region_y=11.2486):
    nodi_scelti=[start_node, end_node]
    percorso_tempo_batt_eur3=euristica3Knn(G,Gsub_D,nodi_scelti,batteria)
    len_percorso=len(percorso_tempo_batt_eur3[(start_node, end_node)])
    len_path=len(percorso_tempo_batt_eur3[(start_node, end_node)][2])
    time_distance=percorso_tempo_batt_eur3[(start_node, end_node)][0]
    len_btw_node=[]
    #battery=percorso_tempo_batt_eur3[(start_node,end_node)][1]
    print("time of travel is : " ,round(time_distance,2)," second")
    #print("battry of travel is : " ,battery," kw/h")
    consumption=0
    distance=0
    tosca_map = folium.Map(location=[region_x,region_y])
    unique_nodes=[]        
    nodes_3=percorso_tempo_batt_eur3[(start_node, end_node)][3]
    nodes_2=percorso_tempo_batt_eur3[(start_node, end_node)][2]
    if percorso_tempo_batt_eur3[(start_node, end_node)][len_percorso-1]=="charge":
        for nnd in nodes_3:
            for j in range(len(nnd)-1):
                distance+=G[nnd[j]][nnd[j+1]][0]['length']
                len_btw_node.append(G[nnd[j]][nnd[j+1]][0]['length'])
                consumption+=float(G[nnd[j]][nnd[j+1]][0]['consumption'])    
        print("distance is: ",round(distance/1000,2)," km")    
        print("consumption ",round((consumption/3600*1e3)*1e-6,2) ," kw")        
        print("stop path is : ",nodes_2)
        #show_plot(len_btw_node)
        counter=0
        for node_list in nodes_3:
            for node in node_list:
                if node in nodes_2 and node not in unique_nodes:
                    if node == nodes_2[0] or node == nodes_2[len_path-1] or node == nodes_2[len_path-2]:
                        clr='green'
                    else:
                        print("charge station node id : ",node ," and type of it's connection is :")
                        node_con = ast.literal_eval(G.nodes[node]['conInf']) 
                        for key,value in node_con.items():
                            print(node_con[key]['Level']['Title'])
                        clr='red'
                        counter=counter+1
                else:
                    clr='green'
                unique_nodes.append(node)
                folium.Marker(location=[G.nodes[node]['y'],G.nodes[node]['x']],icon=folium.Icon(color=clr)).add_to(tosca_map)
        print("user has to charge",counter,"times on the way")
    else:
        k=0
        for nnd1 in nodes_2:
            if k+1 >= len(nodes_2):
                break
            distance+=G[nnd1][nodes_2[k+1]][0]['length']
            consumption+=float(G[nnd1][nodes_2[k+1]][0]['consumption'])  
            k+=1
        print("distance is: ",round(distance/1000,2)," km")    
        print("consumption ",round((consumption/3600*1e3)*1e-6,2)," kw") 
        print("there is no need to charge")
        nodes=percorso_tempo_batt_eur3[(start_node,end_node)][2]
        for node in nodes:
            folium.Marker(location=[G.nodes[node]['y'],G.nodes[node]['x']],icon=folium.Icon(color='green')).add_to(tosca_map)

        #waiting_time=waiting_time_in_charging_station(eBatCap,start_node,end_node,batteria=None)   
        #print(waiting_time)
        
    return tosca_map,len_btw_node


In [ ]:
def waiting_time_in_charging_station_for_test_cons(start_node,end_node,batteria=None):
    global list_of_stops
    nodi_scelti=[start_node, end_node]
    percorso_tempo_batt_eur3=euristica3Knn(G,Gsub_D,nodi_scelti,batteria)
    len_list=len(percorso_tempo_batt_eur3[(start_node, end_node)])
    battery_amount_destination=dict()
    distance=0
    consumption=0
    dic_result=dict()
    i=0
    if percorso_tempo_batt_eur3[(start_node, end_node)][len_list-1]=="charge":     
        list_of_stops=percorso_tempo_batt_eur3[(start_node, end_node)][2]
        #print(list_of_stops)
        len_stops=len(list_of_stops)
        for node_ids in list_of_stops:
            if i+1 >= len_stops:
                break
            node_stops=[list_of_stops[i],list_of_stops[i+1]]
            info_way=euristica3Knn(G,Gsub_D,node_stops,eBatCap)
            nodes_path=info_way[tuple(node_stops)][2]
            k=0
            for nnd1 in nodes_path:
                if k+1 >= len(nodes_path):
                    break
                #print(G[nnd1][nodes_path[k+1]][0])
                if 'length' in G[nnd1][nodes_path[k+1]][0]:
                    distance+=G[nnd1][nodes_path[k+1]][0]['length']
                if 'consumption' in G[nnd1][nodes_path[k+1]][0]:
                    consumption+=float(G[nnd1][nodes_path[k+1]][0]['consumption']) 
                    
                    
                if node_ids == list_of_stops[0] or node_ids == list_of_stops[len_stops-1] or node_ids == list_of_stops[len_stops-2]:
                    dic_result[tuple(node_stops)]={"distance":distance/1000,"consumption":(consumption/3600*1e3)*1e-6,"battery":info_way[tuple(node_stops)][1]}
                else:
                    res = ast.literal_eval(G.nodes[node_ids]['conInf']) 
                    for key,value in res.items():
                        dic_result[tuple(node_stops)]={"distance":distance/1000,"consumption":(consumption/3600*1e3)*1e-6,"battery":info_way[tuple(node_stops)][1],"conInf":res[key]['Level']['Title']}
                k+=1   
            #battery_amount_destination[tuple(node_stops)]={"battery":info_way[tuple(node_stops)][1]}
            distance=0
            consumption=0
            i+=1
    return dic_result,list_of_stops
   

In [ ]:
def test_consumption(number_of_test,battery):
    start_node_list=[]
    end_node_list=[]
    list_of_difference=[]
    counter_corr=0
    counter_d=0
    for i in range(number_of_test):
        start_node, end_node = random.sample(G.nodes(), 2)
        start_node_list.append(start_node)
        end_node_list.append(end_node)
    for j in range(len(start_node_list)):
        try:
            result_dict,list_of_stops=waiting_time_in_charging_station_for_test_cons(start_node_list[j],end_node_list[j],battery)
            first_consumption=result_dict[(list_of_stops[0],list_of_stops[1])]['consumption']
            if first_consumption>battery:
                difference=first_consumption-battery
                list_of_difference.append(difference)
                counter_d+=1
                #print("the battery that we pass is :",battery," the consumtion from start node to first stop is :",first_consumption," the difference is : ",difference)
            else:
                counter_corr+=1
                #print("correct situation")
        except (TypeError,KeyError):
            continue
            
    mean_d = mean(list_of_difference)
    median_d = median(list_of_difference)
    min_d=min(list_of_difference)
    max_d=max(list_of_difference)
    print("number of correct sitation : ",counter_corr)
    print("number of different sitation : ",counter_d)
    print("list of difference : ",list_of_difference)
    print("mean : "+str(mean_d))
    print("median : "+str(median_d))
    print("min : "+str(min_d))
    print("max : "+str(max_d))
            


In [ ]:
def simulation_traj(user_id,m_id):
    input_filename = 'GZIPPED-JSON-FILE-WITH-IMNS' # TODO: replace with file name
    data = gzip.GzipFile(input_filename, 'r')
    start_li=[]
    end_li=[]
    k=0
    test_map = folium.Map(location=[43.7711,11.2486])
    for row in data:
        user_data = json.loads(row)
        if user_data['uid']==user_id:
            for mid,value in user_data['01-02']['movement_prototype'].items():  
                print(mid,user_data['01-02']['movement_prototype'][mid]['_length'])
                if mid==m_id:
                    #print("length : ",user_data['03-04']['movement_prototype'][mid]['_length'])
                    mov_obj=user_data['01-02']['movement_prototype'][mid]['object']
                    len_mov=len(mov_obj)
                    if k > len_mov-1:
                        break
                    for mov in mov_obj:
                        folium.Marker(location=[mov[1],mov[0]],icon=folium.Icon(color='green')).add_to(test_map)
                        if k==0:
                            start_point=(mov[1],mov[0])
                            start_point_osm_id = ox.get_nearest_node(G, start_point)
                            start_li.append(start_point_osm_id)
                        if k==len_mov-1:
                            end_point=(mov[1],mov[0])
                            end_point_osm_id = ox.get_nearest_node(G, end_point)
                            end_li.append(end_point_osm_id)
                        k+=1
            print("start nearest node in G : ",start_point_osm_id)
            print("end nearest node in G : ",end_point_osm_id)


    return test_map


In [ ]:
def movement_ids_of_user(user_id):
    list_of_mid=list()
    input_filename = 'GZIPPED-JSON-FILE-WITH-IMNS' # TODO: replace with file name
    data = gzip.GzipFile(input_filename, 'r')
    for row in data:
        user_data = json.loads(row)
        if user_data['uid']==user_id:
            for mid,value in user_data['movement_prototype'].items():  
                list_of_mid.append(mid)
    return list_of_mid
    

In [ ]:
#type charge 1 is slow 2 is medium and 3 is fast
def time_charge_per_kw(value,battery_need):
    min_wait=0
    if 'Level 2' in value:
        #it is fast charge which is 4 hours for filling 40kw so it is 240 min and 6 min per kw
        min_wait=6
        
    elif 'Level 1' in value:
        #it is slow charge which is 6 hours for filling 40kw so it is 360 min and 9 min per kw
        min_wait=9
        
    elif 'Level 3' in value:
        #it is fast charge which charge 80% in 30 min so 1.06 per kw from 0 to 32kw and from 32 to 40 1.5 min 
        if battery_need < 32:
            min_wait=1.06
        else :
            min_wait=1.5
        
    return min_wait 

In [ ]:
def over_charge(consumption,battery_capacity,overcharge_list):
    if consumption > battery_capacity:
                    over_limit=consumption-battery_capacity
                    over_limit=round(over_limit,2)
                    overcharge_list.append(over_limit)
                    print("you have ",over_limit ," kw over charge")
    

In [ ]:
def full_charge(dic_result,batteria,battery_capacity=40):
    
    amount_of_battery_full=list()
    overcharge_list_full=list()
    main_info_list_full=list()
    l=0
    last_cons_full=0
    for key1,value1 in dic_result.items(): 
        if 'conInf' not in dic_result[key1]:
            last_cons_full+=dic_result[key1]['consumption']
        if 'conInf' in value1:
            if l==0:
                cons_confidence=batteria+8
                #cons_confidence=batteria
                cons_confidence=cons_confidence-value1['consumption']
                battery_need_for_full=battery_capacity-cons_confidence
            else:
                over_charge(value1['consumption'],battery_capacity,overcharge_list_full) 
                new_cons=battery_capacity-value1['consumption']
                battery_need_for_full=battery_capacity-new_cons  
            min_wait=time_charge_per_kw(value1['conInf'],battery_need_for_full)
            time_for_charge_in_min=battery_need_for_full*min_wait
            main_info_list_full.append([key1[1],round(battery_need_for_full,2),round(time_for_charge_in_min,2),value1['traveltime']])
            #print("you need to charge",round(battery_need_for_full,2),"kw for being full and you have to wait",round(time_for_charge_in_min,2),"minutes at charging station",key1[1],"travel time is ",value1['traveltime'])                
            amount_of_battery_full.append(battery_need_for_full)
            l+=1
    arrival_battery=40-last_cons_full
    if not overcharge_list_full:
        overcharge_list_full=[]
        
    return main_info_list_full,arrival_battery
                
def partial_charge(dic_result,batteria,battery_need_for_next_trip=0,battery_capacity=40):
    batteria=batteria+8
    ll=0
    list_of_dict_keys=list()
    time_of_waiting_partial=list()
    main_info_list_partial=list()
    cons_in_last_charge_station=0
    count_conf=0
    #special_case_count=0
    overcharge_list_partial=list()
    
    for k,v in dic_result.items(): 
        list_of_dict_keys.append(k)
    reverse_key_list=list(reversed(list_of_dict_keys)) 
    
    for key1 in reverse_key_list:
        if 'conInf' in dic_result[key1]:
            count_conf+=1
        if 'conInf' not in dic_result[key1]:
            cons_in_last_charge_station=cons_in_last_charge_station+dic_result[key1]['consumption']
    cons_in_last_charge_station=cons_in_last_charge_station+battery_need_for_next_trip

            
    #print("number of charge stations :",count_conf)
    for key1 in reverse_key_list:
        if 'conInf' in dic_result[key1]:
            if ll==0 and count_conf>1:
                over_charge(cons_in_last_charge_station,battery_capacity,overcharge_list_partial) 
                min_wait=time_charge_per_kw(dic_result[key1]['conInf'],cons_in_last_charge_station)
                time_for_charge_in_min_final=cons_in_last_charge_station*min_wait
                #print("you need to charge",round(cons_in_last_charge_station,2),"kw and you have to wait",round(time_for_charge_in_min_final,2),"minutes at charging station",key1[1])
                main_info_list_partial.append([key1[1],round(cons_in_last_charge_station,2),round(time_for_charge_in_min_final,2)])
                cons_in_last_charge_station=dic_result[key1]['consumption']

               
            elif count_conf>2 and ll!=count_conf-1:
                over_charge(cons_in_last_charge_station,battery_capacity,overcharge_list_partial) 
                min_wait=time_charge_per_kw(dic_result[key1]['conInf'],cons_in_last_charge_station)
                time_for_charge_in_min=cons_in_last_charge_station*min_wait
                #print("you need to charge",round(cons_in_last_charge_station,2),"kw and you have to wait",round(time_for_charge_in_min,2),"minutes at charging station",key1[1])
                main_info_list_partial.append([key1[1],round(cons_in_last_charge_station,2),round(time_for_charge_in_min,2)])
                cons_in_last_charge_station=dic_result[key1]['consumption']
       
            if ll==count_conf-1:
                battery_level=batteria-dic_result[key1]['consumption']
                if battery_level > cons_in_last_charge_station:
                    #print("special case")
                    charge_in_first_charge_station=0
                else:
                    over_charge(cons_in_last_charge_station,battery_capacity,overcharge_list_partial) 
                    charge_in_first_charge_station=cons_in_last_charge_station-battery_level
                    min_wait=time_charge_per_kw(dic_result[key1]['conInf'],charge_in_first_charge_station)
                    time_for_charge_in_min=charge_in_first_charge_station*min_wait
                    main_info_list_partial.append([key1[1],round(charge_in_first_charge_station,2),round(time_for_charge_in_min,2)])
                    #print("you need to charge",round(charge_in_first_charge_station,2),"kw and you have to wait",round(time_for_charge_in_min,2),"minutes at charging station",key1[1])
            ll+=1
        
    if not overcharge_list_partial:
        overcharge_list_partial=[]
    return main_info_list_partial
    

In [ ]:
def charging_info_at_charging_station(start_node,end_node,batteria=None,battery_need_for_next_trip=0,battery_cap=40,charge_condition_full=True,charge_condition_partial=True,just_dict=False):
    total_waiting_time_full=0
    total_waiting_time_partial=0
    overcharge_list_full=list()
    overcharge_list_partial=list()
    list_of_stops=list()
    battery_amount_destination=dict()
    distance=0
    consumption=0
    traveltime=0
    dic_result=dict()
    last_cons_full=0
    i=0   
    nodi_scelti=[start_node, end_node]
    
    percorso_tempo_batt_eur3=euristica3Knn(G,Gsub_D,nodi_scelti,batteria)
    
    len_list=len(percorso_tempo_batt_eur3[(start_node, end_node)])
    if percorso_tempo_batt_eur3[(start_node, end_node)][len_list-1]=="charge":     
        list_of_stops=percorso_tempo_batt_eur3[(start_node, end_node)][2]
        #print(list_of_stops)
        len_stops=len(list_of_stops)
        for node_ids in list_of_stops:
            if i+1 >= len_stops:
                break
            node_stops=[list_of_stops[i],list_of_stops[i+1]]
            info_way=euristica3Knn(G,Gsub_D,node_stops,eBatCap)
            nodes_path=info_way[tuple(node_stops)][2]
            k=0
            for nnd1 in nodes_path:
                if k+1 >= len(nodes_path):
                    break
                
                if 'traveltime' in G[nnd1][nodes_path[k+1]][0]:
                    traveltime+=float(G[nnd1][nodes_path[k+1]][0]['traveltime'])
                if 'length' in G[nnd1][nodes_path[k+1]][0]:
                    distance+=G[nnd1][nodes_path[k+1]][0]['length']
                if 'consumption' in G[nnd1][nodes_path[k+1]][0]:
                    consumption+=float(G[nnd1][nodes_path[k+1]][0]['consumption']) 
                    #print("cons between",nnd1,nodes_path[k+1],(consumption/3600*1e3)*1e-6)
                if node_stops[1] == list_of_stops[0] or node_stops[1]  == list_of_stops[len_stops-1] or node_stops[1]  == list_of_stops[len_stops-2]:
                    dic_result[tuple(node_stops)]={"distance":distance/1000,"consumption":(consumption/3600*1e3)*1e-6,"traveltime":traveltime}
                else:
                    res = ast.literal_eval(G.nodes[node_stops[1]]['conInf']) 
                    for key,value in res.items():
                        dic_result[tuple(node_stops)]={"distance":distance/1000,"consumption":(consumption/3600*1e3)*1e-6,"traveltime":traveltime,"conInf":res[key]['Level']['Title']}
                k+=1   
            distance=0
            consumption=0
            i+=1
    if just_dict:
        return dic_result
    if charge_condition_full:   
        #print("for full charge:")
        main_info_list_full,last_cons_full=full_charge(dic_result,batteria,battery_cap)
        #print("-----------------------------------------------------------------------")
    if charge_condition_partial:
        #print("for partial charge:")
        main_info_list_partial=partial_charge(dic_result,batteria,battery_need_for_next_trip)
    
    
    if charge_condition_full:
        return dic_result,main_info_list_full,last_cons_full
    if charge_condition_partial:
        return dic_result,main_info_list_partial
    if charge_condition_full and charge_condition_partial:
        return dic_result,main_info_list_full,main_info_list_partial,last_cons_full


In [ ]:
def test_random_waiting_time(start_node,end_node,battery,battery_need_for_next_trip=0,battery_cap=40,charge_condition_full=True,charge_condition_partial=True,just_dict=False):
    # TEST FUNCTION
    dic,list_of_stops,total_full,total_partial,over_full,over_partial,last_cons_full=charging_info_at_charging_station(start_node,end_node,battery,battery_need_for_next_trip,battery_cap=40,charge_condition_full=True,charge_condition_partial=True,just_dict=False)
    total_full=round(total_full,2)
    total_partial=round(total_partial,2)
    diff_time=round(total_full-total_partial,2)
    return diff_time,total_full,total_partial,over_full,over_partial,dic,last_cons_full

In [ ]:
def find_start_end_osm_nodes(user_id,m_id,date):
    input_filename = 'GZIPPED-JSON-FILE-WITH-IMNS' # TODO: replace with file name
    data = gzip.GzipFile(input_filename, 'r')
    start_li=[]
    end_li=[]
    k=0
    start_point_osm_id=0
    end_point_osm_id=0
    for row in data:
        user_data = json.loads(row)
        if user_data['uid']==user_id:
            print(user_data['regular_locations'])
            for mid,value in user_data['movement_prototype'].items():  
                if mid==m_id:
                    if (date in user_data['movement_prototype'][mid]['_start_time']) and (date in  user_data['movement_prototype'][mid]['_end_time']):
                        print(user_data['movement_prototype'][mid]['_start_time'],"--",user_data['movement_prototype'][mid]['_end_time'])
                        mov_obj=user_data['movement_prototype'][mid]['object']
                        len_mov=len(mov_obj)
                        if k > len_mov-1:
                            break
                        for mov in mov_obj:
                            if k==0:
                                start_point=(mov[1],mov[0])
                                start_point_osm_id = ox.get_nearest_node(G, start_point)
                                start_li.append(start_point_osm_id)
                            if k==len_mov-1:
                                end_point=(mov[1],mov[0])
                                end_point_osm_id = ox.get_nearest_node(G, end_point)
                                end_li.append(end_point_osm_id)
                            k+=1
                    else:
                        continue
    return start_point_osm_id,end_point_osm_id


In [ ]:
def test_users_for_waiting_time_charging_station(user_id):
    # TEST FUNCTION
    list_of_mid=movement_ids_of_user(user_id)
    #list_of_mid=['17','18','20']
    counter=0
    list_of_diff=list()
    start_list=list()
    end_list=list()
    
    for mid in list_of_mid:
        start,end=find_start_end_osm_nodes(user_id,mid,'2017-01-02')
        if start==0 and end==0:
            continue
        print(mid,start,end)
        start_list.append(start)
        end_list.append(end)
        len_end=len(end_list)
    print(start_list)
    print(end_list)
    print("aaaaaa",start_list[0],end_list[len_end-1])
    diff,total_full,total_partial,over_full,over_partial=test_random_waiting_time(start_list[0],end_list[len_end-1],5)
    if diff==0:
        print("no need to charge")
    return diff,total_full,total_partial,over_full,over_partial

In [ ]:
def test_users_for_waiting_time_charging_station_1(cur, uid, min_length, battery, table_name='tak.italy_traj'):
    # TEST FUNCTION
    list_of_diff=list()
    list_of_partial=list()
    list_of_full=list()
    list_over_full=list()
    list_over_partial=list()
    user_traj=db.load_individual_mobility_history(cur, uid, table_name,min_length)
    for key,value in user_traj.items():
        if isinstance(value,dict):
            for key1,value1 in value.items():
                value1=value1.__dict__
                len_value1=len(value1['object'])
                start_cord=value1['object'][0]
                start_point=(start_cord[1],start_cord[0])
                start_node = ox.get_nearest_node(G, start_point)

                end_cord=value1['object'][len_value1-1]
                end_point=(end_cord[1],end_cord[0])
                end_node = ox.get_nearest_node(G, end_point)
                print(start_node,end_node)
                try:
                    diff,total_full,total_partial,over_full,over_partial=test_random_waiting_time(start_node,end_node,battery)
                   
                    #print(diff)
                    if diff<0:
                        print("we have negative!!!!!!!!!!!!!",start_node,end_node)
                        #continue
                        #list_of_negative.append(diff)
                    if diff !=0:
                        list_of_diff.append(diff)
                        list_of_partial.append(total_partial)
                        list_of_full.append(total_full)
                        if over_full:
                            list_over_full.append(over_full)
                        if over_partial:
                            list_over_partial.append(over_partial)
                        
                        
                except(KeyError,TypeError):
                    continue
    if not list_over_full:
        list_over_full=[]
    if not list_over_partial:
        list_over_partial=[]
    return list_of_diff,list_of_partial,list_of_full,list_over_full,list_over_partial